In [ ]:
import ollama
import osmnx as ox
import pandas as pd
import os
import sys

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from routing import get_continuing_road_path, get_turn_path, get_roundabout_path


In [ ]:
INITIAL_PROMPT = INITIAL_PROMPT = """
You are a navigation decision assistant.

You are given:
- The **current road name** (e.g., "High Street"),
- The **next road name** (e.g., "Station Road"),
- The **current node**, which represents the current geographic position in the road graph (this is a node ID from a road network graph),
- A **natural language instruction** (e.g., "At roundabout take the second exit", "Continue straight", "Turn right at the junction").
- There may be extra information in the instruction that is not required, such as nearby landmarks. You will need to filter these.

Your task is to:
1. Choose the correct function to call from the available tools:
   - `get_turn_path` for regular left, right, at standard junctions.
   - `get_roundabout_path` for instructions involving roundabouts (e.g., "Take the third exit").
   - `get_continuing_road_path` when the road simply changes name but no actual turn occurs (e.g., "Continue onto Station Road").

2. Extract all required arguments from the instruction and road names:
   - If the instruction mentions a turn, or a mini roundabout,  extract the direction ("left", "right", "straight").
   - If the instruction involves a roundabout or roundabout abbreviation (RB, R-bout etc), extract the exit number as an integer (e.g., "Take the 2nd exit" → 2).

3. Call exactly **one** of the tools with appropriate arguments.
   - Use the `current_node` as the starting point in all function calls.
   - If none of the tools are suitable, reply with a message explaining why and that the instruction requires human attention.

Be precise and cautious. Only call a function if you're confident it matches the instruction.
If the instruction is ambiguous, unsupported, or incomplete, do **not** guess — respond clearly that it can't be handled automatically.
"""

In [11]:
def run_llm(input: dict, prompt: str, tools: list, model_name: str = "qwen3:8b"):
    """_summary_

    Args:
        input (dict): _description_
        prompt (str): _description_
        model_name (_type_, optional): _description_. Defaults to "qwen3:8b".
    """

    messages = [
        {
            "role": "system",
            "content": prompt
        },
        {
            "role": "user",
            "content": (
                f"You are at node {input['current_node']} on {input['current_road']}.\n"
                f"The next road is {input['next_road']}.\n"
                f"The instruction is: '{input['instruction']}'."
            )
        }
    ]

    try:
        print('Running modle')
        response = ollama.chat(
            model=model_name,
            messages=messages,
            tools=tools, # This is where you pass the tool definitions
        )
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [12]:
node1_2_input = {
    "current_node": "23780711",
    "current_road": "Douglas Road",
    "next_road": "Ewell Road",
    "instruction": "left at EOR"
}

llm_response = run_llm(
    input = node1_2_input, prompt = INITIAL_PROMPT,
                        tools = [get_continuing_road_path, get_turn_path, get_roundabout_path]
                        )


Running modle


In [15]:
node1_2_input = {
    "current_node": "23780711",
    "current_road": "Douglas Road",
    "next_road": "Ewell Road",
    "instruction": "becomes"
}

llm_response_becomes = run_llm(
    input = node1_2_input, prompt = INITIAL_PROMPT,
                        tools = [get_continuing_road_path, get_turn_path, get_roundabout_path]
                        )


Running modle


In [18]:
node1_2_input = {
    "current_node": "23780711",
    "current_road": "Douglas Road",
    "next_road": "Ewell Road",
    "instruction": "at roundabout take 5th exit"
}

llm_response_roundabout = run_llm(
    input = node1_2_input, prompt = INITIAL_PROMPT,
                        tools = [get_continuing_road_path, get_turn_path, get_roundabout_path]
                        )

Running modle


In [19]:
node1_2_input = {
    "current_node": "23780711",
    "current_road": "Douglas Road",
    "next_road": "Ewell Road",
    "instruction": "go backwards then forwards twice"
}

llm_response_notsure = run_llm(
    input = node1_2_input, prompt = INITIAL_PROMPT,
                        tools = [get_continuing_road_path, get_turn_path, get_roundabout_path]
                        )

Running modle


In [17]:
llm_response_becomes.message.tool_calls

[ToolCall(function=Function(name='get_continuing_road_path', arguments={'G': 'OSMNX graph', 'current_node': '23780711', 'current_road_name': 'Douglas Road', 'next_road_name': 'Ewell Road'}))]

In [14]:
llm_response.message.tool_calls

[ToolCall(function=Function(name='get_turn_path', arguments={'G': 'OSMNX_graph', 'current_node': '23780711', 'current_road_name': 'Douglas Road', 'direction': 'left', 'next_road_name': 'Ewell Road'}))]

In [22]:
llm_response_notsure.message.tool_calls

In [21]:
llm_response_roundabout.message.tool_calls

[ToolCall(function=Function(name='get_roundabout_path', arguments={'G': 'OSMNX graph', 'current_node': '23780711', 'current_road_name': 'Douglas Road', 'exit': 5, 'next_road_name': 'Ewell Road'}))]